# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path=os.path.expanduser(r"C:\Users\Zahinos\env"))

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [10]:
context = [{'role': 'system', 'content': """
CREATE TABLE employees (
    ID_usr INTEGER,
    name VARCHAR
);

INSERT INTO employees (ID_usr, name) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie');

CREATE TABLE salary (
    ID_usr INTEGER,
    year DATE,
    salary FLOAT
);

INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2021', 50000),
(2, '2021', 60000),
(1, '2022', 52000);

CREATE TABLE studies (
    ID INTEGER,
    ID_usr INTEGER,
    educational_level INTEGER,
    Institution VARCHAR,
    Years DATE,
    Speciality VARCHAR
);

INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 3, 'MIT', '2018', 'Computer Science'),
(2, 2, 2, 'Harvard', '2019', 'Economics');
"""}]

context.append({'role': 'system', 'content': """
-- Q1: What is the name of the employee with the highest salary?
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;

-- Q2: What institutions have employees studied at?
SELECT DISTINCT Institution FROM studies;

-- Q3: What are the names of employees who studied Computer Science?
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Speciality = 'Computer Science';
"""})




In [11]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [12]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [17]:
#new
context_user = context.copy()
print(return_CCRMSQL("Which employees had a salary higher than 51000 in 2022?", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = '2022' AND s.salary > 51000;
```


In [18]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Which employees had a salary higher than 51000 in 2022?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 51000 AND YEAR(s.year) = 2022;
```

This SQL query retrieves the names of employees who had a salary higher than 51000 in the year 2022 by joining the "employees" and "salary" tables on the employee ID and filtering the results based on the salary and year.


In [19]:
#new
print(return_CCRMSQL("What is the average salary of all employees in 2021?", context_user))

```sql
SELECT AVG(salary) AS average_salary
FROM salary
WHERE year = '2021';
```


In [20]:
#old
print(return_CCRMSQL("What is the average salary of all employees in 2021?", old_context_user))

This is your SQL:
```sql
SELECT AVG(salary) AS average_salary
FROM salary
WHERE year = '2021';
```

This SQL query selects the average salary (`AVG(salary)`) from the `salary` table where the year is 2021.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?